In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.special import i0

In [1]:
print('Lassi')

Lassi


![FORMULA](formula.png) 

In [ ]:
# Quantities declaration
c=300000000
carrier_frequency=30
wavelength=c/(carrier_frequency*(10**9)) #Becuase freq in ghz
number_of_flows=80
number_of_time_slots=2000
slot_duration=0.1
fading_depth=2
bg_noise=-134
bandwidth=2000
transmission_power=40
avg_power_uav=30
peak_power_uav=2*avg_power_uav
transceiver_efficiency=0.8
uav_height=100
uav_speed=20
pl_exponent_v2v=2.5
pl_exponent_u2v=2
rician=9
interference_threshold=1/(10**3)
si_cancellation_level=1/(10**9)
max_gain=20
half_power_beamwidth=30

In [2]:
def calculate_antenna_gain(max_gain: float,direction: float,half_power_beamwidth: float):
  deduction=min((direction/half_power_beamwidth)**2,26)
  gain=max_gain-deduction
  return gain

In [3]:
# g here is channel power gain and m is parameter for Gama pehelwan function
def calculate_channel_power_gain_distribution(g: float,m: int):
  distribution=((1/m)**(-m))*(g**(m-1))*(math.e**(-1*(m*g)))
  distribution/=math.gamma(m)
  return distribution

In [4]:
def calculate_received_signal_power_direct_link(pl_exponent: float,transmission_power: float,max_gain: float,channel_power_gain: float,distance: float):
    kv=(wavelength/(4*math.pi))**pl_exponent
    signal_power=kv*transmission_power*max_gain*channel_power_gain*(distance**(-pl_exponent))
    return signal_power

In [5]:
def calculate_mutual_interference(pl_exponent: float,transmission_power: float,gain: float,channel_power_gain: float,distance: float):
    kv=(wavelength/(4*math.pi))**pl_exponent
    signal_power=kv*transmission_power*gain*channel_power_gain*(distance**(-pl_exponent))
    return signal_power 

In [6]:
# beta= cancellation level
def calculate_rsi(beta: float,transmission_power: float):
    return beta*transmission_power

In [7]:
# n=no of flows, n0 = noise power spectral density, w = channel bandwidth
# eta= efficiency
def calculate_sinr_and_datarate_v2v(n: int,n0: float,w: float,rsi: float,eta: float,received_signal_power,a,b):
    sinr=np.empty(n+1)
    datarate=np.empty(n+1)
    for i in range(1,n+1):
        num=received_signal_power[i][i]
        denom=n0*w
        for j in range(1,n+1):
            if j==i:
                continue
            denom+=(a[i][j]*received_signal_power[j][i])+(b[i][j]*rsi)
        sinr[i]=num/denom
        datarate[i]=eta*w*math.log2(1+sinr[i])
    return sinr,datarate

In [8]:
#k=rician factor
def calculate_non_central_chi_square_distribution(k,total_power,w,i0):
    ans=((k+1)*math.exp(-k))/total_power
    ans*=math.exp((-(k+1)*w)/total_power)
    ans*=i0(2*math.sqrt((k*(k+1)*w)/total_power))
    return ans

In [9]:
def calculate_sinr_and_datarate_u2v(n,pl_exponent,transmit_power_uav,max_gain,distance_u2v,power_fading,n0,eta,w,rsi,received_signal_power,a,b):
    sinr=np.empty(n+1)
    datarate=np.empty(n+1)
    ku=(wavelength/(4*math.pi))**pl_exponent
    for k in range(1,n+1):
        num=ku*transmit_power_uav*max_gain*power_fading*(distance_u2v[k]**(-pl_exponent))
        denom=n0*w
        for i in range(1,n+1):
            if i==k:
                continue
            denom+=(a[k][i]*received_signal_power[i][k])+(b[k][i]*rsi)
        sinr[k]=num/denom
        datarate[k]=eta*w*math.log2(1+sinr[k])
    return sinr,datarate

In [10]:
def calculate_sinr_and_datarate_v2u(n,pl_exponent,transmission_power,max_gain,distance_v2u,power_fading,n0,w,eta):
    sinr=np.empty(n+1)
    datarate=np.empty(n+1)
    kv=(wavelength/(4*math.pi))**pl_exponent
    for k in range(1,n+1):
        num=kv*transmission_power*max_gain*power_fading*(distance_v2u[k]**(-pl_exponent))
        denom=n0*w
        sinr[k]=num/denom
        datarate[k]=eta*w*math.log2(1+sinr[k])
    return sinr,datarate

# Formulating Problem using Proposed Approach